### functon 1 讀取檔案
input ： path, is_display(bool)


output： 物件

In [16]:
import os
from OCC.Display.SimpleGui import init_display
from OCC.Extend.DataExchange import read_iges_file

path = r"C:\NTHU\IRTI-Project\simple_model_2.IGS"  #物件路徑

def read_file(path, is_display=False):
    
    if os.path.exists(path):           # 防止路徑不存在
        shapes = read_iges_file(path)  # 一定要用這個方法讀取
    else:
        print("檔案路徑錯誤！")
    
    if is_display:
        # 初始化 3D 顯示環境
        display, start_display, add_menu, add_function_to_menu = init_display()
        display.DisplayShape(shapes, update=True)
        start_display()
        
    return shapes

In [17]:
object_1 = read_file(path, is_display=1)

### functon 2 分割物件平面
input ： object, save_path, is_save(bool)

output： 面 1 ~ 面 n

In [19]:
import os
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Display.OCCViewer import OffscreenRenderer

# 平面影像儲存路徑
image_directory = r"C:\NTHU\IRTI-Project\Images"

def plane_segmentation(object, save_path, is_save=True):
    
    if not os.path.exists(save_path):        # 如果 save_path 不存在，則創建一個
        os.makedirs(save_path)
        
    
    explorer = TopExp_Explorer(object, TopAbs_FACE) # TopAbs_FACE 只對面感興趣
    faces = []
    index = 1
    # 宣告一個渲彩器(不會顯示出來)
    renderer = OffscreenRenderer()
    renderer.Create()
    
    while explorer.More():
        
        face = topods.Face(explorer.Current())
        faces.append(face)                     # 將當前的面儲存到 list 中
        
        renderer.EraseAll()
        renderer.DisplayShape(face, update=True, color='BLUE', transparency=0.5)  # 可以調整顏色和透明度
        renderer.FitAll()
        
        if is_save:
            image_path = os.path.join(image_directory, f'object_face_{index}.png')            
            renderer.View.Dump(image_path)
            print(f'Saved image of face {index} to {image_path}')

        index += 1
        explorer.Next()                      # 移動到下一個面
    
    return faces

In [20]:
object_1_faces = plane_segmentation(object_1, image_directory)
print("object_1 總共有", len(object_1_faces), "個物件")

Many colors for color name BLUE, using first.
Saved image of face 1 to C:\NTHU\IRTI-Project\Images\object_face_1.png
Many colors for color name BLUE, using first.
Saved image of face 2 to C:\NTHU\IRTI-Project\Images\object_face_2.png
Many colors for color name BLUE, using first.
Saved image of face 3 to C:\NTHU\IRTI-Project\Images\object_face_3.png
Many colors for color name BLUE, using first.
Saved image of face 4 to C:\NTHU\IRTI-Project\Images\object_face_4.png
Many colors for color name BLUE, using first.
Saved image of face 5 to C:\NTHU\IRTI-Project\Images\object_face_5.png
object_1 總共有 5 個物件


### functon 3 平面+線段 生成 drive plane
input ： object, save_path, is_save(bool)

output： 面 1 ~ 面 n